In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from mlxtend.frequent_patterns import apriori, association_rules

In [4]:
pd.set_option('display.max_columns', 70)

In [8]:
df = pd.read_csv('./result/online-retail-clean.csv')

print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397884 entries, 0 to 397883
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   main_idx     397884 non-null  int64  
 1   InvoiceNo    397884 non-null  int64  
 2   StockCode    397884 non-null  object 
 3   Description  397884 non-null  object 
 4   Quantity     397884 non-null  int64  
 5   InvoiceDate  397884 non-null  object 
 6   UnitPrice    397884 non-null  float64
 7   CustomerID   397884 non-null  float64
 8   Country      397884 non-null  object 
dtypes: float64(2), int64(3), object(4)
memory usage: 27.3+ MB
None


,main_idx,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [9]:
# Filter the data into last 3-months
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['date'] = pd.DatetimeIndex(df['InvoiceDate']).date
df['date'] = pd.to_datetime(df['date'])

# Time splitting
n_days = 90
max_date = df['date'].max()
cutoff = max_date - pd.to_timedelta(n_days, unit='d')

df = df[df['InvoiceDate'] > cutoff].reset_index(drop=True)

print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161526 entries, 0 to 161525
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   main_idx     161526 non-null  int64         
 1   InvoiceNo    161526 non-null  int64         
 2   StockCode    161526 non-null  object        
 3   Description  161526 non-null  object        
 4   Quantity     161526 non-null  int64         
 5   InvoiceDate  161526 non-null  datetime64[ns]
 6   UnitPrice    161526 non-null  float64       
 7   CustomerID   161526 non-null  float64       
 8   Country      161526 non-null  object        
 9   date         161526 non-null  datetime64[ns]
dtypes: datetime64[ns](2), float64(2), int64(3), object(3)
memory usage: 12.3+ MB
None


,main_idx,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,date
0,236358,566225,22384,LUNCH BAG PINK POLKADOT,6,2011-09-11 10:35:00,1.65,16899.0,United Kingdom,2011-09-11
1,236359,566225,20725,LUNCH BAG RED RETROSPOT,6,2011-09-11 10:35:00,1.65,16899.0,United Kingdom,2011-09-11
2,236360,566225,23169,CLASSIC GLASS COOKIE JAR,3,2011-09-11 10:35:00,4.15,16899.0,United Kingdom,2011-09-11
3,236361,566225,84029E,RED WOOLLY HOTTIE WHITE HEART.,3,2011-09-11 10:35:00,4.25,16899.0,United Kingdom,2011-09-11
4,236362,566225,21479,WHITE SKULL HOT WATER BOTTLE,3,2011-09-11 10:35:00,4.25,16899.0,United Kingdom,2011-09-11


In [360]:
# # Sampling data
# import random

# sample_size = 1000
# idx_data = []
# boundary = []
# for i in range(1, 4):
#     upper = max_date - pd.to_timedelta((i - 1)*30, unit='d')
#     lower = max_date - pd.to_timedelta(i * 30, unit='d')
#     boundary.append((lower, upper))
#     data = list(df[(df['InvoiceDate'] <= upper) & (df['InvoiceDate'] > lower)].index)
#     random_sample = random.sample(data, sample_size)
#     idx_data = idx_data + random_sample

# print(len(idx_data))

In [361]:
# sample_data = df.iloc[idx_data]
# sample_data.head()

In [10]:
basket = df.groupby('InvoiceNo').agg({'Description': set, 'InvoiceDate': 'first'}).reset_index()
basket

,InvoiceNo,Description,InvoiceDate
0,566225,"{PACK OF 60 MUSHROOM CAKE CASES, CLASSIC GLASS...",2011-09-11 10:35:00
1,566226,"{DOLLY MIXTURE CHILDREN'S UMBRELLA, RED RETROS...",2011-09-11 10:39:00
2,566227,"{JUMBO BAG PINK POLKADOT, EDWARDIAN PARASOL NA...",2011-09-11 10:42:00
3,566228,"{JUMBO BAG 50'S CHRISTMAS , HAND WARMER SCOTTY...",2011-09-11 10:52:00
4,566229,"{HEART MIRROR ANTIQUE WHITE, LAVENDER TOILETTE...",2011-09-11 10:53:00
...,...,...,...
6621,581583,"{6 CHOCOLATE LOVE HEART T-LIGHTS, LUNCH BAG RE...",2011-12-09 12:23:00
6622,581584,"{6 CHOCOLATE LOVE HEART T-LIGHTS, RED FLOCK LO...",2011-12-09 12:25:00
6623,581585,"{ALARM CLOCK BAKELIKE RED , BOX OF 24 COCKTAIL...",2011-12-09 12:31:00
6624,581586,"{DOORMAT RED RETROSPOT, SET OF 3 HANGING OWLS ...",2011-12-09 12:49:00


In [363]:
# # Sampling data
# import random

# sample_size = 1000
# idx_data = []
# boundary = []
# for i in range(1, 4):
#     upper = max_date - pd.to_timedelta((i - 1)*30, unit='d')
#     lower = max_date - pd.to_timedelta(i * 30, unit='d')
#     boundary.append((lower, upper))
#     data = list(basket[(basket['InvoiceDate'] <= upper) & (basket['InvoiceDate'] > lower)].index)
#     random_sample = random.sample(data, sample_size)
#     idx_data = idx_data + random_sample

# print(len(idx_data))

In [364]:
# sample_basket = basket.iloc[idx_data]
# sample_basket

In [11]:
basket_encoded = basket['Description'].str.join('|').str.get_dummies('|')
basket_encoded                                                      

,4 PURPLE FLOCK DINNER CANDLES,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,I LOVE LONDON MINI RUCKSACK,NINE DRAWER OFFICE TIDY,OVAL WALL MIRROR DIAMANTE,RED SPOT GIFT BAG LARGE,SPACEBOY BABY GIFT SET,TRELLIS COAT RACK,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 DAISY PEGS IN WOOD BOX,12 EGG HOUSE PAINTED WOOD,12 HANGING EGGS HAND PAINTED,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE SKULLS,12 PENCILS TALL TUBE WOODLAND,12 RED ROSE PEG PLACE SETTINGS,15 PINK FLUFFY CHICKS IN BOX,15CM CHRISTMAS GLASS BALL 20 LIGHTS,16 PIECE CUTLERY SET PANTRY DESIGN,18PC WOODEN CUTLERY SET DISPOSABLE,2 DAISIES HAIR COMB,2 PICTURE BOOK EGGS EASTER BUNNY,2 PICTURE BOOK EGGS EASTER CHICKS,2 PICTURE BOOK EGGS EASTER DUCKS,20 DOLLY PEGS RETROSPOT,200 BENDY SKULL STRAWS,...,WRAP PINK FAIRY CAKES,WRAP POPPIES DESIGN,WRAP RED APPLES,WRAP RED VINTAGE DOILY,WRAP SUKI AND FRIENDS,WRAP VINTAGE LEAF DESIGN,WRAP WEDDING DAY,YELLOW COAT RACK PARIS FASHION,YELLOW DRAGONFLY HELICOPTER,YELLOW GIANT GARDEN THERMOMETER,YELLOW POT PLANT CANDLE,YELLOW SHARK HELICOPTER,YELLOW/BLUE RETRO RADIO,YELLOW/PINK FLOWER DESIGN BIG MUG,YOU'RE CONFUSING ME METAL SIGN,YULETIDE IMAGES GIFT WRAP SET,ZINC HEART T-LIGHT HOLDER,ZINC STAR T-LIGHT HOLDER,ZINC BOX SIGN HOME,ZINC FOLKART SLEIGH BELLS,ZINC HEART FLOWER T-LIGHT HOLDER,ZINC HEART LATTICE CHARGER LARGE,ZINC HEART LATTICE CHARGER SMALL,ZINC HEART LATTICE T-LIGHT HOLDER,ZINC HEARTS PLANT POT HOLDER,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC SWEETHEART SOAP DISH,ZINC SWEETHEART WIRE LETTER RACK,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC TOP 2 DOOR WOODEN SHELF,ZINC WILLIE WINKIE CANDLE STICK,ZINC WIRE KITCHEN ORGANISER,ZINC WIRE SWEETHEART LETTER TRAY
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6621,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6622,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6623,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
6624,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [366]:
# basket_encoded = sample_basket['Description'].str.join('|').str.get_dummies('|')
# basket_encoded                                                      

In [12]:
# Convert the DataFrame to use boolean types
basket_encoded_bool = basket_encoded.astype(bool)

# Apply Apriori algorithm
frequent_itemsets = apriori(basket_encoded_bool, 
                            min_support=0.01, 
                            use_colnames=True)

# Display frequent itemsets
frequent_itemsets

,support,itemsets
0,0.015545,( 50'S CHRISTMAS GIFT BAG LARGE)
1,0.011168,( DOLLY GIRL BEAKER)
2,0.010263,( I LOVE LONDON MINI BACKPACK)
3,0.012225,( RED SPOT GIFT BAG LARGE)
4,0.013734,(10 COLOUR SPACEBOY PEN)
...,...,...
1402,0.012074,"(HOT WATER BOTTLE I AM SO POORLY, CHOCOLATE HO..."
1403,0.010414,"(GREEN REGENCY TEACUP AND SAUCER, ROSES REGENC..."
1404,0.011621,"(HAND WARMER SCOTTY DOG DESIGN, HAND WARMER OW..."
1405,0.010414,"(HAND WARMER SCOTTY DOG DESIGN, HAND WARMER UN..."


In [13]:
# Generate association rules with lower lift threshold

rules = association_rules(frequent_itemsets, metric='lift', min_threshold=1)

rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(12 PENCIL SMALL TUBE WOODLAND),(SET 12 COLOUR PENCILS SPACEBOY ),0.025506,0.034863,0.011168,0.437870,12.559850,0.010279,1.716929,0.944470
1,(SET 12 COLOUR PENCILS SPACEBOY ),(12 PENCIL SMALL TUBE WOODLAND),0.034863,0.025506,0.011168,0.320346,12.559850,0.010279,1.433810,0.953627
2,(WOOD 2 DRAWER CABINET WHITE FINISH),(3 DRAWER ANTIQUE WHITE WOOD CABINET),0.028977,0.025506,0.014941,0.515625,20.216161,0.014202,2.011859,0.978900
3,(3 DRAWER ANTIQUE WHITE WOOD CABINET),(WOOD 2 DRAWER CABINET WHITE FINISH),0.025506,0.028977,0.014941,0.585799,20.216161,0.014202,2.344328,0.975413
4,(6 GIFT TAGS 50'S CHRISTMAS ),(6 GIFT TAGS VINTAGE CHRISTMAS ),0.036221,0.034863,0.016903,0.466667,13.385859,0.015640,1.809633,0.960069
...,...,...,...,...,...,...,...,...,...,...
1781,"(SET OF 3 WOODEN TREE DECORATIONS, SET OF 3 WO...","(SET OF 3 WOODEN HEART DECORATIONS, SET OF 3 W...",0.025204,0.019771,0.014187,0.562874,28.470266,0.013688,2.242443,0.989823
1782,(SET OF 3 WOODEN HEART DECORATIONS),"(SET OF 3 WOODEN TREE DECORATIONS, SET OF 3 WO...",0.042711,0.017507,0.014187,0.332155,18.972950,0.013439,1.471141,0.989558
1783,(SET OF 3 WOODEN SLEIGH DECORATIONS),"(SET OF 3 WOODEN HEART DECORATIONS, SET OF 3 W...",0.032448,0.019620,0.014187,0.437209,22.284222,0.013550,1.741998,0.987156
1784,(SET OF 3 WOODEN TREE DECORATIONS),"(SET OF 3 WOODEN HEART DECORATIONS, SET OF 3 W...",0.033203,0.015998,0.014187,0.427273,26.708576,0.013655,1.718099,0.995616


In [14]:
# Display association rules
rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head(10)

,antecedents,consequents,support,confidence,lift
0,(12 PENCIL SMALL TUBE WOODLAND),(SET 12 COLOUR PENCILS SPACEBOY ),0.011168,0.437870,12.559850
1,(SET 12 COLOUR PENCILS SPACEBOY ),(12 PENCIL SMALL TUBE WOODLAND),0.011168,0.320346,12.559850
2,(WOOD 2 DRAWER CABINET WHITE FINISH),(3 DRAWER ANTIQUE WHITE WOOD CABINET),0.014941,0.515625,20.216161
3,(3 DRAWER ANTIQUE WHITE WOOD CABINET),(WOOD 2 DRAWER CABINET WHITE FINISH),0.014941,0.585799,20.216161
4,(6 GIFT TAGS 50'S CHRISTMAS ),(6 GIFT TAGS VINTAGE CHRISTMAS ),0.016903,0.466667,13.385859
5,(6 GIFT TAGS VINTAGE CHRISTMAS ),(6 GIFT TAGS 50'S CHRISTMAS ),0.016903,0.484848,13.385859
6,(6 GIFT TAGS 50'S CHRISTMAS ),(BOX OF 6 MINI 50'S CRACKERS),0.010564,0.291667,5.910041
7,(BOX OF 6 MINI 50'S CRACKERS),(6 GIFT TAGS 50'S CHRISTMAS ),0.010564,0.214067,5.910041
8,(6 GIFT TAGS 50'S CHRISTMAS ),(PAPER CHAIN KIT 50'S CHRISTMAS ),0.014187,0.391667,3.374751
9,(PAPER CHAIN KIT 50'S CHRISTMAS ),(6 GIFT TAGS 50'S CHRISTMAS ),0.014187,0.122237,3.374751


In [15]:
rules['filter_1'] = rules['antecedents'].apply(lambda x: 1 if len(x) < 2 else 0)
rules['filter_2'] = rules['consequents'].apply(lambda x: 1 if len(x) < 2 else 0)

final = rules[(rules['filter_1'] == 1) & (rules['filter_2'] == 1)]\
        .drop(['filter_1', 'filter_2'], axis=1).copy()

In [16]:
final['antecedents'] = final.loc[:,'antecedents'].apply(lambda x: next(iter(x)))
final['consequents'] = final.loc[:,'consequents'].apply(lambda x: next(iter(x)))
final = final[final['confidence'] > 0.5].reset_index(drop=True)
final.to_csv('./result/association-analysis.csv', index_label='idx_assc')
final.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,WOOD 2 DRAWER CABINET WHITE FINISH,3 DRAWER ANTIQUE WHITE WOOD CABINET,0.028977,0.025506,0.014941,0.515625,20.216161,0.014202,2.011859,0.978900
1,3 DRAWER ANTIQUE WHITE WOOD CABINET,WOOD 2 DRAWER CABINET WHITE FINISH,0.025506,0.028977,0.014941,0.585799,20.216161,0.014202,2.344328,0.975413
2,ALARM CLOCK BAKELIKE CHOCOLATE,ALARM CLOCK BAKELIKE GREEN,0.020223,0.052068,0.014639,0.723881,13.902704,0.013586,3.433052,0.947228
3,ALARM CLOCK BAKELIKE CHOCOLATE,ALARM CLOCK BAKELIKE PINK,0.020223,0.039390,0.011168,0.552239,14.019672,0.010372,2.145362,0.947840
4,ALARM CLOCK BAKELIKE CHOCOLATE,ALARM CLOCK BAKELIKE RED,0.020223,0.050709,0.013734,0.679104,13.392102,0.012708,2.958255,0.944429


In [17]:
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173 entries, 0 to 172
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   antecedents         173 non-null    object 
 1   consequents         173 non-null    object 
 2   antecedent support  173 non-null    float64
 3   consequent support  173 non-null    float64
 4   support             173 non-null    float64
 5   confidence          173 non-null    float64
 6   lift                173 non-null    float64
 7   leverage            173 non-null    float64
 8   conviction          173 non-null    float64
 9   zhangs_metric       173 non-null    float64
dtypes: float64(8), object(2)
memory usage: 13.6+ KB


In [18]:
item = np.sort(final['antecedents'].unique())
print(len(item))
# item

126


In [374]:
source_target = final[['antecedents', 'consequents']].values

Q = nx.Graph()
Q.add_edges_from(source_target)
coordinates = nx.spring_layout(Q)

coordinates

{'WOOD 2 DRAWER CABINET WHITE FINISH': array([0.60242601, 0.67477263]),
 '3 DRAWER ANTIQUE WHITE WOOD CABINET': array([0.57872446, 0.68538033]),
 'ALARM CLOCK BAKELIKE CHOCOLATE': array([ 0.54115553, -0.42184868]),
 'ALARM CLOCK BAKELIKE GREEN': array([ 0.53181069, -0.4450861 ]),
 'ALARM CLOCK BAKELIKE PINK': array([ 0.51291602, -0.4218825 ]),
 'ALARM CLOCK BAKELIKE RED ': array([ 0.51564962, -0.43766619]),
 'ALARM CLOCK BAKELIKE IVORY': array([ 0.50537395, -0.45552563]),
 'PAINTED METAL PEARS ASSORTED': array([0.33471139, 0.78558738]),
 'ASSORTED COLOUR BIRD ORNAMENT': array([0.31737007, 0.74296888]),
 'BAKING SET SPACEBOY DESIGN': array([0.44690009, 0.78279959]),
 'BAKING SET 9 PIECE RETROSPOT ': array([0.43372936, 0.75606559]),
 'LE JARDIN BOTANIQUE CUSHION COVER': array([-0.76691521, -0.23539857]),
 'BELLE JARDINIERE CUSHION COVER': array([-0.81434671, -0.24821485]),
 'CLASSIC BICYCLE CLIPS ': array([-0.44387276, -0.79866264]),
 'BICYCLE PUNCTURE REPAIR KIT ': array([-0.47011444, -

In [375]:
df_coordinates = pd.DataFrame(coordinates).T
df_coordinates.columns = ['X', 'Y']
df_coordinates.to_csv('./result/CoorFile.csv', index_label='NodeName')
df_coordinates.head()

,X,Y
WOOD 2 DRAWER CABINET WHITE FINISH,0.602426,0.674773
3 DRAWER ANTIQUE WHITE WOOD CABINET,0.578724,0.685380
ALARM CLOCK BAKELIKE CHOCOLATE,0.541156,-0.421849
ALARM CLOCK BAKELIKE GREEN,0.531811,-0.445086
ALARM CLOCK BAKELIKE PINK,0.512916,-0.421883


In [376]:
src_target = source_target.reshape(len(source_target) * 2)
print(src_target.shape)
df_nodes = pd.DataFrame(src_target, columns=['NodeName'])


array_idx = []
for i in range(1, len(src_target) + 1):
    array_idx.append((i + 1) // 2)
    
print(len(array_idx))
df_nodes['c_index'] = array_idx
df_nodes.to_csv('./result/BridgeFile.csv', index_label='idxNodes')
df_nodes.head(6)

(346,)
346


,NodeName,c_index
0,WOOD 2 DRAWER CABINET WHITE FINISH,1
1,3 DRAWER ANTIQUE WHITE WOOD CABINET,1
2,3 DRAWER ANTIQUE WHITE WOOD CABINET,2
3,WOOD 2 DRAWER CABINET WHITE FINISH,2
4,ALARM CLOCK BAKELIKE CHOCOLATE,3
5,ALARM CLOCK BAKELIKE GREEN,3


In [377]:
df_nodes.tail(6)

,NodeName,c_index
340,WOODEN TREE CHRISTMAS SCANDINAVIAN,171
341,WOODEN HEART CHRISTMAS SCANDINAVIAN,171
342,WOODEN STAR CHRISTMAS SCANDINAVIAN,172
343,WOODEN TREE CHRISTMAS SCANDINAVIAN,172
344,WOODEN TREE CHRISTMAS SCANDINAVIAN,173
345,WOODEN STAR CHRISTMAS SCANDINAVIAN,173


In [389]:
final[final['consequents'] == 'RED RETROSPOT CHARLOTTE BAG']

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
19,CHARLOTTE BAG APPLES DESIGN,RED RETROSPOT CHARLOTTE BAG,0.020072,0.039088,0.011319,0.563910,14.426510,0.010534,2.203470,0.949747
20,CHARLOTTE BAG PINK POLKADOT,RED RETROSPOT CHARLOTTE BAG,0.022185,0.039088,0.016149,0.727891,18.621648,0.015281,3.531350,0.967769
21,CHARLOTTE BAG SUKI DESIGN,RED RETROSPOT CHARLOTTE BAG,0.025657,0.039088,0.013281,0.517647,13.242971,0.012278,1.992134,0.948832
22,CHARLOTTE BAG VINTAGE ALPHABET,RED RETROSPOT CHARLOTTE BAG,0.018714,0.039088,0.010263,0.548387,14.029393,0.009531,2.127733,0.946433
119,STRAWBERRY CHARLOTTE BAG,RED RETROSPOT CHARLOTTE BAG,0.019016,0.039088,0.012526,0.658730,16.852301,0.011783,2.815694,0.958895
120,WOODLAND CHARLOTTE BAG,RED RETROSPOT CHARLOTTE BAG,0.023242,0.039088,0.013432,0.577922,14.784987,0.012523,2.276621,0.954549


In [393]:
len(np.unique(unique_floats))

125

In [394]:
len(unique_floats)

125